In [2]:
import pandas as pd
import requests
import numpy as np
import requests
import bs4
import re
from datetime import datetime
import csv
import os

In [3]:
movies= pd.read_csv('IMDB movies.csv', low_memory=False)
currency= pd.read_csv('Currency.csv', low_memory=False)

In [4]:
budgetmovies = movies[movies['budget'].notnull()].copy()
budgetmovies['currency'] = budgetmovies.loc[:,'budget'].str.slice(0,3)
budgetmovies['currency'] = budgetmovies['currency'].apply(lambda x: 'USD' if '$' in x else x)
budgetmovies.head()

,Unnamed: 0,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Currency,currency
1,2,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0,USD,USD
3,4,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0,USD,USD
7,8,tt0002445,Quo Vadis?,Quo Vadis?,1913,1913-03-01,"Drama, History",120,Italy,Italian,...,6.2,273,ITL 45000,NaN,NaN,NaN,7.0,5.0,ITL,ITL
8,9,tt0002452,Independenta Romaniei,Independenta Romaniei,1912,1912-09-01,"History, War",120,Romania,NaN,...,6.7,198,ROL 400000,NaN,NaN,NaN,4.0,1.0,ROL,ROL
9,10,tt0002461,Richard III,Richard III,1912,1912-10-15,Drama,55,"France, USA",English,...,5.5,225,$ 30000,NaN,NaN,NaN,8.0,1.0,USD,USD


In [5]:
print(budgetmovies["worlwide_gross_income"].isna().sum().sum())
print(budgetmovies["usa_gross_income"].isna().sum().sum())
usamovies = budgetmovies[budgetmovies["usa_gross_income"].notnull()].copy()
print(len(usamovies))

10948
15584
8126


In [30]:
usamovies1 = usamovies[usamovies["country"].notnull()].copy()
usamovies2 = usamovies1[usamovies1["country"].str.match("USA")]
usamovies3 = usamovies[usamovies["currency"].str.match("USD")]
print(len(usamovies3))
print(len(usamovies2))


7281
5571


In [32]:
usamovies2.head(50)

,Unnamed: 0,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Currency,currency
245,246,tt0012190,I quattro cavalieri dell'Apocalisse,The Four Horsemen of the Apocalypse,1921,1923-04-16,"Drama, Romance, War",150,USA,None,...,7.2,3058,$ 800000,$ 9183673,$ 9183673,NaN,45.0,16.0,USD,USD
1048,1049,tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,...,8.5,162668,$ 1500000,$ 19181,$ 46008,99.0,295.0,122.0,USD,USD
2454,2455,tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,...,8.5,211250,$ 1500000,$ 163577,$ 457688,96.0,307.0,115.0,USD,USD
2827,2828,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,...,7.6,177157,$ 1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,USD,USD
3266,3267,tt0031381,Via col vento,Gone with the Wind,1939,1949-03-12,"Drama, History, Romance",238,USA,English,...,8.1,283975,$ 3977000,$ 200852579,$ 402352579,97.0,881.0,197.0,USD,USD
3348,3349,tt0031679,Mr. Smith va a Washington,Mr. Smith Goes to Washington,1939,1947-04-05,"Comedy, Drama",129,USA,English,...,8.1,104547,$ 1900000,$ 144738,$ 144738,73.0,296.0,88.0,USD,USD
3451,3452,tt0032138,Il mago di Oz,The Wizard of Oz,1939,1949-04-19,"Adventure, Family, Fantasy",102,USA,English,...,8.0,366293,$ 2777000,$ 24790250,$ 26142032,100.0,688.0,168.0,USD,USD
3553,3554,tt0032455,Fantasia,Fantasia,1940,1946-09-19,"Animation, Family, Fantasy",125,USA,English,...,7.8,86795,$ 2280000,$ 76408097,$ 76411401,96.0,342.0,119.0,USD,USD
3672,3673,tt0032910,Pinocchio,Pinocchio,1940,1947-11-27,"Animation, Comedy, Family",88,USA,English,...,7.4,127618,$ 2600000,$ 84254167,$ 121892045,99.0,202.0,140.0,USD,USD
3816,3817,tt0033467,Quarto potere,Citizen Kane,1941,1948-11-25,"Drama, Mystery",119,USA,"English, Italian",...,8.3,389322,$ 839727,$ 1585634,$ 1594107,100.0,1452.0,248.0,USD,USD


In [31]:
usamovies3.head(50)

,Unnamed: 0,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Currency,currency
165,166,tt0010323,Il gabinetto del dottor Caligari,Das Cabinet des Dr. Caligari,1920,1920-02-27,"Fantasy, Horror, Mystery",76,Germany,German,...,8.1,55601,$ 18000,$ 8811,$ 8811,NaN,237.0,160.0,USD,USD
245,246,tt0012190,I quattro cavalieri dell'Apocalisse,The Four Horsemen of the Apocalypse,1921,1923-04-16,"Drama, Romance, War",150,USA,None,...,7.2,3058,$ 800000,$ 9183673,$ 9183673,NaN,45.0,16.0,USD,USD
1048,1049,tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,...,8.5,162668,$ 1500000,$ 19181,$ 46008,99.0,295.0,122.0,USD,USD
2454,2455,tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,...,8.5,211250,$ 1500000,$ 163577,$ 457688,96.0,307.0,115.0,USD,USD
2795,2796,tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937-10-22,"Crime, Drama, Romance",94,France,"French, Arabic",...,7.7,6180,$ 60000,$ 155895,$ 155895,98.0,46.0,55.0,USD,USD
2827,2828,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,...,7.6,177157,$ 1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,USD,USD
3266,3267,tt0031381,Via col vento,Gone with the Wind,1939,1949-03-12,"Drama, History, Romance",238,USA,English,...,8.1,283975,$ 3977000,$ 200852579,$ 402352579,97.0,881.0,197.0,USD,USD
3348,3349,tt0031679,Mr. Smith va a Washington,Mr. Smith Goes to Washington,1939,1947-04-05,"Comedy, Drama",129,USA,English,...,8.1,104547,$ 1900000,$ 144738,$ 144738,73.0,296.0,88.0,USD,USD
3451,3452,tt0032138,Il mago di Oz,The Wizard of Oz,1939,1949-04-19,"Adventure, Family, Fantasy",102,USA,English,...,8.0,366293,$ 2777000,$ 24790250,$ 26142032,100.0,688.0,168.0,USD,USD
3553,3554,tt0032455,Fantasia,Fantasia,1940,1946-09-19,"Animation, Family, Fantasy",125,USA,English,...,7.8,86795,$ 2280000,$ 76408097,$ 76411401,96.0,342.0,119.0,USD,USD


In [ ]:
#possible way to merge but will have to go line by line
NYUSCounties=pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

NYALCounties=NYUSCounties[NYUSCounties['state']=="Alabama"]
NYALCounties = NYALCounties.sort_values(['county', 'date'])
NYALCounties=NYALCounties.reset_index(drop=True)
NYALCounties['date'] = pd.to_datetime(NYALCounties['date'])
NYALCounties['date'] = NYALCounties['date'].dt.date

#Pivoting to calculated diff and fill na-- for Daily Cases
sorted_data = NYALCounties.pivot(index = 'date', columns = 'county', values = 'cases')
sorted_data = sorted_data.fillna(0)
for col in sorted_data.columns:
    sorted_data[col] = sorted_data[col].diff()
sorted_data = sorted_data.reset_index()
#reformating data to be able to merge
melted_data = pd.melt(sorted_data, id_vars = 'date')
melted_data = melted_data.fillna(0)
melted_data = melted_data.rename(columns = {'value': 'Daily_Cases'})
#merging Daily Cases into dataframe
NYALCounties = NYALCounties.merge(melted_data, how = 'left')